# Testing disease prevalence as explanatory variable for death rate

## Code book

### For Covid-19 data

#### Variables and their units

- We have 2 dataframes (df_infected, df_deaths) each one contains only one type of information
   - Infected
   - Deaths
- dates (columns): days of the year 2020 from end of January to today
- deaths (entries of df_deaths): # of people died of covid-19
- Infected (entries of df_infected): # of people tested positive on covid-19
- Countries (index): Names of all Countries that maintain a record of their covid-19 cases


#### Summary choices made

- We made up a new column for Continents Names of all Continents 
- We calculated a death rate for the latest day and each country by computing deaths divided by infected
        

### For diseases data

#### Variables and their units

- Our dataframe contained:
    - Entity -> Country names
    - Country 3 letter code
    - year
    - about 20 diseases each reporting in % how large the proportion of deaths is caused by this disease respect to total deaths in a country in a year

#### Summary choices made

- We discarded many diseases and all years from the past (we have chosen 2016 as recent year since it's complete unlike 2017)
- the diseases we investigated are reported as risk factors for covid-19 and are the following:
    - Cardiovascular diseases
    - Diabetes
    - Respiratory diseases
    - Nutritional deficiencies


### Plots
- We  made scatterplots with x-axis the prevelance of a disease vs. the deathrate and hoped to see any correlation, but we didn't
- a linear regression was performed for europe with respiratory diseases but the residuals and the metrics don't look good



## Import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pycountry_convert as pc

## Read in data

Data from: https://github.com/CSSEGISandData/COVID-19

In [2]:
#import data of deaths and infected
deaths = pd.read_csv('./COVID-19-master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
infected = pd.read_csv('./COVID-19-master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths_US = pd.read_csv('./COVID-19-master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
infected_US = pd.read_csv('./COVID-19-master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
#add a column called United States to the US datasets so that they are compatible
deaths_US['Country/Region'] = 'United States'
infected_US['Country/Region'] = 'United States'

data from: https://ourworldindata.org/grapher/share-of-total-disease-burden-by-cause

In [3]:
diseases = pd.read_csv('./diseases.csv')

## Data wrangling

In [4]:
#statistics for countries (group all regions together)
deaths = deaths.groupby('Country/Region').sum() 
infected = infected.groupby('Country/Region').sum() 
#sum over all states in the US
deaths_US = deaths_US.groupby('Country/Region').sum()
infected_US = infected_US.groupby('Country/Region').sum()

In [5]:
#take just the most recent day (i.e. data) for analysis
deaths = deaths[deaths.columns[-1]]
infected = infected[infected.columns[-1]]
deaths_US = deaths_US[deaths_US.columns[-1]]
infected_US = infected_US[infected_US.columns[-1]]
deaths = deaths.append(deaths_US)
infected = infected.append(infected_US)

In [6]:
#create a pd.series with contains death_rates for all countries within the datasets
death_rate = deaths/infected
countries_death_rate = death_rate.index

In [7]:
diseases.columns

Index(['Entity', 'Code', 'Year', 'Cardiovascular diseases (%)', 'Cancers (%)',
       'Diabetes, blood, & endocrine diseases (%)',
       'Mental and substance use disorders (%)', 'Respiratory diseases (%)',
       'Other NCDs (%)', 'Liver disease (%)',
       'Diarrhea & common infectious diseases (%)', 'Neonatal disorders (%)',
       'Maternal disorders (%)', 'Musculoskeletal disorders (%)',
       'Unintentional injuries (%)', 'Neurological disorders (%)',
       'HIV/AIDS and tuberculosis (%)', 'Transport injuries (%)',
       'Self-harm (%)', 'Interpersonal violence (%)',
       'Conflict & terrorism (%)', 'Natural disasters (%)',
       'Malaria & neglected tropical diseases (%)',
       'Nutritional deficiencies (%)', 'Digestive diseases (%)',
       'Other communicable diseases (%)'],
      dtype='object')

### interesting variables
- Cardiovascular diseases
- Diabetes
- Respiratory diseases
- Nutritional deficiencies

In [8]:
diseases = diseases.loc[(diseases['Year'] == 2016)]
#have to take 2016 since for 2017 there is no data available regarding diabetes

In [9]:
diseases

,Entity,Code,Year,Cardiovascular diseases (%),Cancers (%),"Diabetes, blood, & endocrine diseases (%)",Mental and substance use disorders (%),Respiratory diseases (%),Other NCDs (%),Liver disease (%),...,HIV/AIDS and tuberculosis (%),Transport injuries (%),Self-harm (%),Interpersonal violence (%),Conflict & terrorism (%),Natural disasters (%),Malaria & neglected tropical diseases (%),Nutritional deficiencies (%),Digestive diseases (%),Other communicable diseases (%)
26,Afghanistan,AFG,2016,8.953006,3.372082,5.273280,2.858358,1.963170,8.610303,0.498116,...,2.938014,3.091958,0.477499,1.364792,11.950411,0.113785,1.393345,1.770030,1.919848,1.318737
54,Albania,ALB,2016,27.016784,13.225390,4.571592,5.021419,3.251425,4.906855,1.122277,...,0.051632,2.559099,0.960000,0.657873,0.089445,0.012838,0.019658,0.980691,2.993068,0.256763
82,Algeria,DZA,2016,17.005476,6.481729,8.614000,7.428263,3.475417,9.213593,0.926308,...,0.390351,4.754736,0.937858,0.379171,0.187965,0.065546,0.124510,1.188142,2.755848,0.597224
110,American Samoa,ASM,2016,17.731443,10.766780,16.741250,4.579241,5.447381,5.238647,1.317823,...,0.832989,2.039153,1.165921,1.292605,0.021500,0.136717,0.299595,1.717747,2.856487,1.455184
138,Andean Latin America,NaN,2016,8.975195,9.638716,6.775455,6.345378,3.097143,6.264238,2.045797,...,1.517249,4.230945,1.129436,1.660275,0.113817,0.309585,0.912056,2.031825,4.376244,0.503100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6354,Western Sub-Saharan Africa,NaN,2016,3.724172,2.791558,3.214157,2.222745,1.278003,5.317264,1.075991,...,7.584554,1.556068,0.380757,0.565460,0.206688,0.004924,12.399159,4.473840,2.317353,1.489165
6382,World,OWID_WRL,2016,14.490059,9.232437,5.586523,4.822218,4.437275,4.923699,1.632587,...,4.237291,3.039545,1.357773,1.018319,0.476779,0.044277,2.520883,2.353481,3.365841,0.982084
6410,Yemen,YEM,2016,11.662122,3.194844,4.071575,4.190816,2.032312,7.713454,0.509678,...,0.657666,5.155841,0.662017,0.991902,8.732391,0.040894,0.910300,7.359534,2.110756,2.998945
6438,Zambia,ZMB,2016,4.074966,3.864070,2.823180,2.743615,1.453882,6.395929,1.369252,...,22.644122,1.749760,0.621454,0.879693,0.003851,0.000432,4.989169,4.257678,2.606263,1.526627


In [10]:
#select columns we investigate
diseases = diseases[['Entity', 'Year', 'Cardiovascular diseases (%)',
       'Diabetes, blood, & endocrine diseases (%)', 'Respiratory diseases (%)','Nutritional deficiencies (%)']]

In [11]:
diseases

,Entity,Year,Cardiovascular diseases (%),"Diabetes, blood, & endocrine diseases (%)",Respiratory diseases (%),Nutritional deficiencies (%)
26,Afghanistan,2016,8.953006,5.273280,1.963170,1.770030
54,Albania,2016,27.016784,4.571592,3.251425,0.980691
82,Algeria,2016,17.005476,8.614000,3.475417,1.188142
110,American Samoa,2016,17.731443,16.741250,5.447381,1.717747
138,Andean Latin America,2016,8.975195,6.775455,3.097143,2.031825
...,...,...,...,...,...,...
6354,Western Sub-Saharan Africa,2016,3.724172,3.214157,1.278003,4.473840
6382,World,2016,14.490059,5.586523,4.437275,2.353481
6410,Yemen,2016,11.662122,4.071575,2.032312,7.359534
6438,Zambia,2016,4.074966,2.823180,1.453882,4.257678


In [12]:
country_names_diseases = np.ndarray.tolist(diseases['Entity'].unique())

In [13]:
#loop through data and create new df that contains only these elementes
#that are part of death_rate.index 

final_diseases = pd.DataFrame()
for index, row in diseases.iterrows():
    if row['Entity'] in countries_death_rate:
          final_diseases = final_diseases.append(row)
            

In [14]:
final_diseases

,Cardiovascular diseases (%),"Diabetes, blood, & endocrine diseases (%)",Entity,Nutritional deficiencies (%),Respiratory diseases (%),Year
26,8.953006,5.273280,Afghanistan,1.770030,1.963170,2016.0
54,27.016784,4.571592,Albania,0.980691,3.251425,2016.0
82,17.005476,8.614000,Algeria,1.188142,3.475417,2016.0
166,13.784688,4.381889,Andorra,0.340551,4.771667,2016.0
194,4.480300,2.733065,Angola,5.597667,1.711981,2016.0
...,...,...,...,...,...,...
6242,12.951494,8.146976,Venezuela,0.672916,2.856847,2016.0
6270,16.842832,6.516006,Vietnam,0.897179,4.246777,2016.0
6410,11.662122,4.071575,Yemen,7.359534,2.032312,2016.0
6438,4.074966,2.823180,Zambia,4.257678,1.453882,2016.0


In [15]:
len(death_rate)
#we have to diminish death_rate series

189

In [16]:
#make the Location to the index, so that we can easily sort the dataframe and get the death_rate we want 
final_diseases.set_index('Entity', inplace=True)

In [17]:
death_rate = death_rate.get(list(final_diseases.index))
#get only the countries where we have lifeexpectancy data


#sort dataframe and series so that we can easily append each to another
final_diseases = final_diseases.sort_index()
death_rate = death_rate.sort_index()

In [18]:
final_diseases['death_rate'] = death_rate

In [19]:
final_diseases

,Cardiovascular diseases (%),"Diabetes, blood, & endocrine diseases (%)",Nutritional deficiencies (%),Respiratory diseases (%),Year,death_rate
Entity,,,,,,
Afghanistan,8.953006,5.273280,1.770030,1.963170,2016.0,0.020601
Albania,27.016784,4.571592,0.980691,3.251425,2016.0,0.032064
Algeria,17.005476,8.614000,1.188142,3.475417,2016.0,0.072237
Andorra,13.784688,4.381889,0.340551,4.771667,2016.0,0.066929
Angola,4.480300,2.733065,5.597667,1.711981,2016.0,0.057971
...,...,...,...,...,...,...
Venezuela,12.951494,8.146976,0.672916,2.856847,2016.0,0.008921
Vietnam,16.842832,6.516006,0.897179,4.246777,2016.0,0.000000
Yemen,11.662122,4.071575,7.359534,2.032312,2016.0,0.189189


In [20]:
countries = np.asarray(list(final_diseases.index))
# Continent_code to Continent_names as a dictionary
continents = {
    'NA': 'North America',
    'SA': 'South America', 
    'AS': 'Asia',
    'OC': 'Australia',
    'AF': 'Africa',
    'EU' : 'Europe',
    'na' : 'Others'
}

# Defining Function for getting continent code for country by using the pc library
def country_to_continent_code(country):
    try:
        return pc.country_alpha2_to_continent_code(pc.country_name_to_country_alpha2(country))
    except :
        return 'na'

#Collecting continent Information, insert at position 1, with column name 'continent'
final_diseases.insert(1,"continent", [continents[country_to_continent_code(country)] for country in countries[:]])

In [21]:
#"Cote d'Ivoire" as others, but is africa
final_diseases.replace('Others', 'Africa', inplace=True)

In [22]:
final_diseases

,Cardiovascular diseases (%),continent,"Diabetes, blood, & endocrine diseases (%)",Nutritional deficiencies (%),Respiratory diseases (%),Year,death_rate
Entity,,,,,,,
Afghanistan,8.953006,Asia,5.273280,1.770030,1.963170,2016.0,0.020601
Albania,27.016784,Europe,4.571592,0.980691,3.251425,2016.0,0.032064
Algeria,17.005476,Africa,8.614000,1.188142,3.475417,2016.0,0.072237
Andorra,13.784688,Europe,4.381889,0.340551,4.771667,2016.0,0.066929
Angola,4.480300,Africa,2.733065,5.597667,1.711981,2016.0,0.057971
...,...,...,...,...,...,...,...
Venezuela,12.951494,South America,8.146976,0.672916,2.856847,2016.0,0.008921
Vietnam,16.842832,Asia,6.516006,0.897179,4.246777,2016.0,0.000000
Yemen,11.662122,Asia,4.071575,7.359534,2.032312,2016.0,0.189189


## Scatterplots
- Cardiovascular diseases vs. death_rate
- Diabetes, blood & endocrine diseases vs. death_rate
- nutrional deficiencies vs. death_rate
- respiratory diseaes vs. death_rate

In [23]:
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import chart_studio
import chart_studio.plotly as py
import plotly.graph_objects as go
import plotly.express as px

In [24]:
chart_studio.tools.set_credentials_file(username='asrira', api_key='u1AQVbhfs7neeJ0TPl1f')

In [25]:
#Jupyter setup (to display in notebook)
init_notebook_mode(connected=True)

In [26]:
countries = list(final_diseases.index)

In [27]:
fig = px.scatter(final_diseases,
    x = 'Cardiovascular diseases (%)',
    y = 'death_rate',
    hover_name=countries,
    color='continent',
    title='Cardiovascular diseases vs Death rate')
fig.update_layout(
  xaxis_title="percentage of deaths attributable to cardiovascular diseases",
  yaxis_title="Death rate")
#fig.show()
#add to plotly account
py.iplot(fig, filename='Cardiovascular diseases', auto_open=True) 

In [28]:
fig = px.scatter(final_diseases,
    x = 'Diabetes, blood, & endocrine diseases (%)',
    y = 'death_rate',
    hover_name=countries,
    color='continent',
    title='Diabetes, blood, & endocrine diseases vs Death rate')
fig.update_layout(
  xaxis_title= "percentage of deaths attributable to diabetes, blood & endocrine diseases",
  yaxis_title="Death rate")
#fig.show()
#add to plotly account
py.iplot(fig, filename='Diabetes, blood, & endocrine diseases', auto_open=True) 

In [29]:
fig = px.scatter(final_diseases,
    x = 'Nutritional deficiencies (%)',
    y = 'death_rate',
    hover_name=countries,
    color='continent',
    title='Nutritional deficiencies vs Death rate')
fig.update_layout(
  xaxis_title= "percentage of deaths attributable to nutritional deficiencies",
  yaxis_title="Death rate")
#fig.show()
#add to plotly account
py.iplot(fig, filename='Nutritional deficiencies', auto_open=True) 

In [30]:
#if one focusses on europe there is at least a weak positive correlation
fig = px.scatter(final_diseases,
    x = 'Respiratory diseases (%)',
    y = 'death_rate',
    hover_name=countries,
    color='continent',
    title='Respiratory diseases vs Death rate')
fig.update_layout(
  xaxis_title="percentage of deaths attributable to respiratory diseases",
  yaxis_title="Death rate")
#fig.show()
#add to plotly account
py.iplot(fig, filename='Respiratory_diseases', auto_open=True) 

## Statistical Inference

In [31]:
europe_data = final_diseases[final_diseases["continent" ]== 'Europe']

In [32]:
europe_countries = list(europe_data.index)

In [33]:
europe_data=europe_data.rename(columns = {'Respiratory diseases (%)':'Respiratory_disease_perc'})

In [34]:
import statsmodels.formula.api as sm
reg = sm.ols(formula='death_rate ~ Respiratory_disease_perc',data=europe_data).fit()
reg.summary()
#get the values of the regression line
europe_data['fitted_values'] = reg.fittedvalues

In [35]:
# plotly figure setup
fig=go.Figure()
fig.add_trace(go.Scatter(name='Respiratory Diseases (%)',x=europe_data['Respiratory_disease_perc'], y=europe_data['death_rate'], hovertext=europe_countries,mode='markers'))
fig.add_trace(go.Scatter(name='line of best fit', x=europe_data['Respiratory_disease_perc'], y=europe_data['fitted_values'], mode='lines'))


# plotly figure layout
fig.update_layout(xaxis_title = 'Respiratory Diseases in %', yaxis_title = 'Death rate', title='Respiratory diseases vs Death rate with Linear Regression')

fig.show()
#add to plotly account
#py.iplot(fig, filename='Respiratory_diseases_Linear_Regression', auto_open=True) 

### Look at residuals

In [36]:
x=europe_data['Respiratory_disease_perc']
fig = go.Figure()
fig.add_trace(go.Scatter(name='Residuals', x=x, y=europe_data['fitted_values']-europe_data['death_rate'], hovertext=europe_countries,mode='markers'))
fig.add_trace(go.Line(name='0',x=x, y=np.zeros_like(x)))
fig.update_layout(xaxis_title = 'Respiratory Diseases in %', yaxis_title = 'Residual', title='Respiratory diseases vs Death rate Residuals')

fig.show()

/opt/anaconda3/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


